# Explicit Modeling

Create a valid boundary representation with the necessary geometrical and topological information can be a complex and time consuming task.
Geode-solutions provides key functionalities in a module named [Geode-Explicit](https://docs.geode-solutions.com/references/geode-explicit/).

The module allows to build "valid" boudary representations directly from a 3D surfaces (or 2D lines) representing boundaries and internal structures of a model.

It involves a robust and inovative technology handling numerical precision error during the computation intersections between lines and surfaces.

In this tutorial, you will learn to build easily boundary representations using the Geode-Explicit python api.

At the end, you will be able to 
 * build a section from a set of 2D edged curves
 * build a BRep from a set of triangulated surfaces

In [ ]:
import opengeode as og 
import opengeode_io 
import opengeode_inspector as inspector
import geode_explicit as explicit
import geode_common as common
import geode_simplex as simplex

print("Geode module imported!")
delta = 0.001

# define various utilities functions
import os
def input(path):
  return os.path.join("data/",path) 

def create_folder_if_does_not_exists(path):
  if( not os.path.isdir(path)):
    os.makedirs(path)

def output(path):
  result_path = "results/"
  create_folder_if_does_not_exists(result_path)
  return os.path.join(result_path,path)

# define print section description
def print_section_components(section) : 
    bb =og.BoundingBox2D()
    bb.add_box(section.bounding_box())
    print("The section is centered on (" + 
          str(bb.center().value(0))+" , "+str(bb.center().value(1)) + 
          "). It is a "+str(bb.diagonal().value(0))+"m by "+
          str(bb.diagonal().value(1))+
          "m section with: ")
    print(" * "+ str(section.nb_corners()) +" corners" )
    for corner in section.corners():
        print("  - corner "+ corner.id().string()+" have " + str(corner.mesh().nb_vertices())+" vertices.") 
    print(" * "+ str(section.nb_lines()) +" lines" )
    for line in section.lines():
        print("  - line "+ line.id().string()+" have " + str(line.mesh().nb_edges())+" edges.") 
    print(" * "+ str(section.nb_surfaces()) +" surfaces" )
    for surface in section.surfaces():
        print("  - surface "+ surface.id().string()+" have " + str(surface.mesh().nb_polygons())+" triangles.") 

# define print section description
def print_brep_components(brep) : 
    bb =og.BoundingBox3D()
    bb.add_box(brep.bounding_box())
    print("The BRep is centered on (" + 
          str(bb.center().value(0))+" , "+str(bb.center().value(1)) + 
          "). It is a "+str(bb.diagonal().value(0))+"m by "+
          str(bb.diagonal().value(1))+
          "m BRep with: ")
    print(" * "+ str(brep.nb_corners()) +" corners" )
    for corner in brep.corners():
        print("  - corner "+ corner.id().string()+" have " + str(corner.mesh().nb_vertices())+" vertices.") 
    print(" * "+ str(brep.nb_lines()) +" lines" )
    for line in brep.lines():
        print("  - line "+ line.id().string()+" have " + str(line.mesh().nb_edges())+" edges.") 
    print(" * "+ str(brep.nb_surfaces()) +" surfaces" )
    for surface in brep.surfaces():
        print("  - surface "+ surface.id().string()+" have " + str(surface.mesh().nb_polygons())+" triangles.") 
    print(" * "+ str(brep.nb_blocks()) +" blocks" )
    for block in brep.blocks():
        print("  - block "+ block.id().string()+" have " + str(block.mesh().nb_polyhedra())+" tetrahedra.") 

## Section from edged curves

We want to build a faulted section defined as follow:

![section from curves](pictures/section_curves.png)

The first step is to define the geometry of section internal and external boundaries by creating edged curves from vertices.

In [ ]:
# Create edged curve from points 
def create_edged_curve(points):
    curve = og.EdgedCurve2D.create()
    builder = og.EdgedCurveBuilder2D.create(curve)
    v0 = builder.create_point(points[0])
    for point in points[1::]:
        v1 = builder.create_point(point)
        builder.create_edge_with_vertices(v0,v1)
        v0 = v1
    return curve

# box geometry
B1 = og.Point2D([0,10])
B2 = og.Point2D([20,10])
B3 = og.Point2D([20,0])
B4 = og.Point2D([0,0])
top_B1B2 = create_edged_curve([B1,B2])
bot_B3B4 = create_edged_curve([B3,B4])
left_B4B1 = create_edged_curve([B4,B1])
right_B3B2 = create_edged_curve([B3,B2])

# horizons geometry
hor_H1H2 = create_edged_curve([og.Point2D([0,6]),og.Point2D([10-delta,6])])
hor_H3H4 = create_edged_curve([og.Point2D([6+delta,3]),og.Point2D([20,3])])

# fault geometry
fault_pts = [og.Point2D([0,0.5]), og.Point2D([1,1]), og.Point2D([6,3]),
             og.Point2D([8,4]), og.Point2D([10,6]), og.Point2D([13,10])]
fault = create_edged_curve(fault_pts)

# save edged curves to be visualized in paraview
og.save_edged_curve2D(top_B1B2,output("top_B1B2.vtp"))
og.save_edged_curve2D(bot_B3B4,output("bot_B3B4.vtp"))
og.save_edged_curve2D(left_B4B1,output("left_B4B1.vtp"))
og.save_edged_curve2D(right_B3B2,output("right_B3B2.vtp"))
og.save_edged_curve2D(hor_H1H2,output("hor_H1H2.vtp"))
og.save_edged_curve2D(hor_H3H4,output("hor_H3H4.vtp"))
og.save_edged_curve2D(fault,output("fault.vtp"))

The geode-explicit module is used to:
 1. compute intersections between edged curves,
 1. create corners, lines and surfaces,
 1. define consistent relationships between corners, lines and surfaces,
 1. define meshes for corners, lines and surfaces,
 1. manage unique vertex identifier.

In [ ]:
voi = og.BoundingBox2D()
voi.add_point(og.Point2D([0,0]))
voi.add_point(og.Point2D([20,10]))

modeler = explicit.SectionExplicitModeler(voi)
modeler.add_curve(top_B1B2)
modeler.add_curve(bot_B3B4)
modeler.add_curve(left_B4B1)
modeler.add_curve(right_B3B2)
modeler.add_curve(hor_H1H2)
modeler.add_curve(hor_H3H4)
modeler.add_curve(fault)

section = modeler.build()
print_section_components(section)

The validity of the section created by the section explicit modeler is guaranteed.

This can be verified using the inspector functionalities.

In [ ]:
# inspector
section_inspector = inspector.SectionInspector(section)
result = section_inspector.inspect_section()
print(result.string())

# save for online inspection
og.save_section(section,output("02-section.og_sctn"))

In [ ]:
# save section to be visualized in paraview
og.save_section(section,output("02-section.vtm"))


However, the section only define two watertight area instead of the four expected. 

![Bad section](pictures/bad_section.png)

This is due to a lack of effective intersections between the horizons and the fault.

In [ ]:
# load section (eventually)
#section = og.load_section(input("02-bad_section.og_sctn"))
#section = og.load_section(input("02-watertight_section.og_sctn"))

#remesh
# set up parameters for homogeneous remeshing
edge_length = 3
cst_metric = common.ConstantMetric2D(edge_length)

# create and return the remeshed section
remeshed_section = simplex.simplex_remesh_section(section,cst_metric)[0]

og.save_section(remeshed_section,output("02-remeshed_section.vtm"))


The section need to be repaired and an effective contact between the horizons and the fault must be defined.

In this particular case, you simply need to set the delta variable initialized in the first cell to 0.0 and re run all cells.

The watertight section can be observed in paraview as:

![Watertight section](pictures/watertight_section.png)

## BRep from triangulated surfaces

In this part, we will learn to:
 1. create planar triangulated surfaces,
 2. use the geode-explicit toolbox in 3D to create valid BRep.

The first step is to learn how to create a planar surface from its orbital points. We called orbital points, the points defining the geometry of surface border (shape of a closed surface). It must defines a polygon with less than 256 points. 

In [ ]:
# create surface from its orbital points in 3D
def create_surface_from_orbital_points(points):
    surface = og.PolygonalSurface3D.create()
    builder = og.PolygonalSurfaceBuilder3D.create(surface)
    vertices=[]
    for point in points:
        vertices.append(builder.create_point(point))
    builder.create_polygon(vertices)
    og.triangulate_surface_mesh3D(surface)
    return og.convert_surface_mesh_into_triangulated_surface3D(surface)

# 4 arbitrary points in defining a square in 3D
points = [
    og.Point3D([0,0,1]),
    og.Point3D([0,1,1]),
    og.Point3D([0,1,0]),
    og.Point3D([0,0,0])
]  

# build 3D triangulated surface 
surface = create_surface_from_orbital_points(points)
print("The surface is a square made of "+ str(surface.nb_polygons()) + " triangles.")

# save surface for paraview 
og.save_triangulated_surface3D(surface,output("surface.vtp"))

The created surface is a square discretized with two triangles and can be observed in paraview. 

![surface](./pictures/surface.png)


### Create BRep of a 3D cube
Start by creating the six surfaces defining the 3D cube aligned on main x,y,z axis. 

In [ ]:
# create an aligned box in 3D
def create_aligned_box3D(extent):
    bbox = og.BoundingBox3D()
    bbox.add_point(og.Point3D([0,0,0]))
    bbox.add_point(og.Point3D([extent,extent,extent]))
    return bbox

# define the alligned box of a particular size
box = create_aligned_box3D(40)

# corner points on the top of the box. 
top1 = og.Point3D([box.min().value(0),box.min().value(1),box.max().value(2)])
top2 = og.Point3D([box.min().value(0),box.max().value(1),box.max().value(2)])
top3 = og.Point3D([box.max().value(0),box.max().value(1),box.max().value(2)])
top4 = og.Point3D([box.max().value(0),box.min().value(1),box.max().value(2)])

# corner points on the bottom of the box. 
bot1 = og.Point3D([box.min().value(0),box.min().value(1),box.min().value(2)])
bot2 = og.Point3D([box.min().value(0),box.max().value(1),box.min().value(2)])
bot3 = og.Point3D([box.max().value(0),box.max().value(1),box.min().value(2)])
bot4 = og.Point3D([box.max().value(0),box.min().value(1),box.min().value(2)])

# surface of the bounding box. 
top = create_surface_from_orbital_points([top1,top2,top3,top4])
bottom = create_surface_from_orbital_points([bot1,bot2,bot3,bot4])
front = create_surface_from_orbital_points([top1,top4,bot4,bot1])
back = create_surface_from_orbital_points([top2,top3,bot3,bot2])
left = create_surface_from_orbital_points([top1,top2,bot2,bot1])
right = create_surface_from_orbital_points([top3,top4,bot4,bot3])

#save surfaces to be vizualized in paraview
og.save_triangulated_surface3D(top,output("top.vtp"))
og.save_triangulated_surface3D(bottom,output("bottom.vtp"))
og.save_triangulated_surface3D(front,output("front.vtp"))
og.save_triangulated_surface3D(back,output("back.vtp"))
og.save_triangulated_surface3D(left,output("left.vtp"))
og.save_triangulated_surface3D(right,output("right.vtp"))

![box3d](./pictures/box3d.png)

The geode-explicit module is used to:
 1. compute intersections between triangulated surfaces (and then between produced edged curves),
 1. create corners, lines, surfaces and blocks,
 1. define consistent relations between corners, lines, surfaces and blocks,
 1. define meshes for corners, lines and surfaces,
 1. manage unique vertex identifier.

In [ ]:
# Use Geode-Explicit to generate the Watertight BRep
modeler = explicit.BRepExplicitModeler(box)
modeler.add_triangulated_surface(top)
modeler.add_triangulated_surface(bottom)
modeler.add_triangulated_surface(front)
modeler.add_triangulated_surface(back)
modeler.add_triangulated_surface(left)
modeler.add_triangulated_surface(right)

brep = modeler.build()
print_brep_components(brep)

og.save_brep(brep,output("02-box_brep.vtm"))
og.save_brep(brep,output("02-box_brep.og_brep"))

The previous steps can be gathered in a single function as follow:

In [ ]:
def create_aligned_boxBRep3D(box):
    # corner points on the top of the box. 
    top1 = og.Point3D([box.min().value(0),box.min().value(1),box.max().value(2)])
    top2 = og.Point3D([box.min().value(0),box.max().value(1),box.max().value(2)])
    top3 = og.Point3D([box.max().value(0),box.max().value(1),box.max().value(2)])
    top4 = og.Point3D([box.max().value(0),box.min().value(1),box.max().value(2)])
    # corner points on the bottom of the box. 
    bot1 = og.Point3D([box.min().value(0),box.min().value(1),box.min().value(2)])
    bot2 = og.Point3D([box.min().value(0),box.max().value(1),box.min().value(2)])
    bot3 = og.Point3D([box.max().value(0),box.max().value(1),box.min().value(2)])
    bot4 = og.Point3D([box.max().value(0),box.min().value(1),box.min().value(2)])
    # surface of the bounding box. 
    top = create_surface_from_orbital_points([top1,top2,top3,top4])
    bottom = create_surface_from_orbital_points([bot1,bot2,bot3,bot4])
    front = create_surface_from_orbital_points([top1,top4,bot4,bot1])
    back = create_surface_from_orbital_points([top2,top3,bot3,bot2])
    left = create_surface_from_orbital_points([top1,top2,bot2,bot1])
    right = create_surface_from_orbital_points([top3,top4,bot4,bot3])
    # explicit modeling to create the BRep
    modeler = explicit.BRepExplicitModeler(box)
    modeler.add_triangulated_surface(top)
    modeler.add_triangulated_surface(bottom)
    modeler.add_triangulated_surface(front)
    modeler.add_triangulated_surface(back)
    modeler.add_triangulated_surface(left)
    modeler.add_triangulated_surface(right)
    return modeler.build()
    
box = create_aligned_box3D(40)
box_brep = create_aligned_boxBRep3D(box)
print_brep_components(box_brep)


Here again, the validity of the BRep created by the BRep explicit modeler is guaranteed and can be verified using the inspector functionalities.

In [ ]:
# inspector
brep_inspector = inspector.BRepInspector(box_brep)
result = brep_inspector.inspect_brep()
print(result.string())

Now it is time to mesh blocks. It works in a very similar way as in 2D.

In [ ]:
# load brep (eventually)
#brbox_brepep = og.load_brep(input("02-box_brep.og_brep"))

# remesh
# set up parameters for homogeneous remeshing
edge_length = 2
cst_metric = common.ConstantMetric3D(edge_length)

# create and return the remeshed section
remeshed_brep = simplex.simplex_remesh_brep(box_brep,cst_metric)[0]

og.save_brep(remeshed_brep,output("02-remeshed_brep.vtm"))
print_brep_components(remeshed_brep)

### Add flat horizontal surfaces in the cubic BRep
Here orbital points are computed using the z position of horizontal surface and the box previously defined.

In [ ]:
def horizontal_surface_orbital_points(box,z_position):
    return[
    og.Point3D([box.min().value(0),box.min().value(1),z_position]),
    og.Point3D([box.min().value(0),box.max().value(1),z_position]),
    og.Point3D([box.max().value(0),box.max().value(1),z_position]),
    og.Point3D([box.max().value(0),box.min().value(1),z_position])]

# load brep (eventually)
#box_brep = og.load_brep(input("02-box_brep.og_brep"))

hor_points = horizontal_surface_orbital_points(box,30)
hor = create_surface_from_orbital_points(hor_points)

# explicit modeling to create the BRep
modeler = explicit.BRepExplicitModeler(box)
modeler.add_brep(box_brep)
modeler.add_triangulated_surface(hor)
layered_brep = modeler.build()

# save  
og.save_brep(layered_brep,output("layered_brep.vtm"))

![surfaces BRep](pictures/surfaces_brep.png)


In [ ]:
# load brep (eventually)
#layered_brep = og.load_brep(input("02-layered_brep.og_brep"))

# remesh
# set up parameters for homogeneous remeshing
edge_length = 2
cst_metric = common.ConstantMetric3D(edge_length)

# create and return the remeshed section
remeshed_layered_brep = simplex.simplex_remesh_brep(layered_brep,cst_metric)[0]

og.save_brep(remeshed_layered_brep,output("02-remeshed_layered_brep.vtm"))
print_brep_components(remeshed_layered_brep)

Please create and visualize several meshes.

![homo mesh brep](pictures/homo_mesh_brep.png)

The mesher allow to create adaptive meshes to bring higher spatial resolution in particular area.

In [ ]:
# load brep (eventually)
#layered_brep = og.load_brep(input("02-layered_brep.og_brep"))

# adaptive remesh
def is_model_boundary(brep_,surface_):
  for col_surf in brep_.collections(surface_.id()):
    if col_surf.type().matches(og.ModelBoundary3D.component_type_static()):
      return True
  return False
  
def small_edges_arround_internal_surface(brep_, default_size_, min_size_):
  metric_constraints = simplex.BRepMetricConstraints(brep_)
  metric_constraints.set_default_metric(default_size_)
  for surf in brep_.surfaces() :
    if(not is_model_boundary(brep_,surf)):
      metric_constraints.set_surface_metric(surf,min_size_)
  return metric_constraints

max_size = 10
min_size = 1
metric_constraints = small_edges_arround_internal_surface(layered_brep,max_size,min_size)
adaptive_brep = simplex.simplex_remesh_brep(layered_brep,metric_constraints.build_metric())[0]

og.save_brep(adaptive_brep,output("02-adaptive_brep.vtm"))

![homo mesh brep](pictures/hetero_mesh_brep.png)

Please try to create several meshes or try to add more surfaces in the model.